In [2]:
# %% [markdown]
# # Food Delivery Hackathon Analysis
# **Author:** Shravani Nigudge  
# **Date:** January 2026  
# 
# This notebook loads transactional, user, and restaurant data, merges them, and answers all hackathon questions.

# %%
# 1️⃣ Import Libraries
import pandas as pd
import sqlite3
import numpy as np

# %%
# 2️⃣ Load Data

# Load orders CSV
orders = pd.read_csv("orders.csv")
orders['order_date'] = pd.to_datetime(orders['order_date'], dayfirst=True)

# Load users JSON
users = pd.read_json("users.json")

# Load restaurants SQL
conn = sqlite3.connect(":memory:")  # in-memory database
with open("restaurants.sql", "r") as f:
    sql_script = f.read()
conn.executescript(sql_script)
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# %%
# 3️⃣ Merge Data

# Merge orders with users
orders_users = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

# Merge with restaurants
final_df = pd.merge(
    orders_users,
    restaurants,
    how="left",
    on="restaurant_id"
)

# Fix column name conflicts
if 'restaurant_name_x' in final_df.columns:
    final_df.rename(columns={'restaurant_name_x': 'restaurant_name'}, inplace=True)

# Save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

# %%
# 4️⃣ Hackathon Questions

# Total orders by Gold members
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]
print("Total orders by Gold members:", gold_orders_count)

# Total revenue in Hyderabad
hyderabad_revenue = round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())
print("Total revenue in Hyderabad:", hyderabad_revenue)

# Distinct users with at least 1 order
distinct_users = final_df['user_id'].nunique()
print("Distinct users with orders:", distinct_users)

# Average order value for Gold members
gold_avg_order = round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(), 2)
print("Average order value for Gold members:", gold_avg_order)

# Orders for restaurants with rating >= 4.5
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]
print("Orders for restaurants with rating >= 4.5:", high_rating_orders)

# Top revenue city among Gold members
top_gold_city = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
top_gold_city_orders = final_df[(final_df['membership'] == 'Gold') & (final_df['city'] == top_gold_city)].shape[0]
print("Top revenue city among Gold members:", top_gold_city)
print("Orders in top revenue city (Gold members):", top_gold_city_orders)

# Revenue by membership + cuisine
combo_revenue = final_df.groupby(['membership','cuisine'])['total_amount'].sum().sort_values(ascending=False)
print("\nRevenue by membership + cuisine combination:")
print(combo_revenue)

# Cuisine with fewest restaurants but significant revenue
cuisine_stats = final_df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id','nunique'),
    total_revenue=('total_amount','sum')
).sort_values(['distinct_restaurants','total_revenue'], ascending=[True, False])
print("\nCuisine stats (fewest restaurants but high revenue):")
print(cuisine_stats)

# Restaurant with highest avg order value but < 20 orders
restaurant_stats = final_df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount','mean'),
    order_count=('order_id','count')
)
high_avg_restaurant = restaurant_stats[restaurant_stats['order_count'] < 20].sort_values('avg_order_value', ascending=False)
print("\nRestaurant with highest avg order value (<20 orders):")
print(high_avg_restaurant.head(5))

# Percentage of orders by Gold members
total_orders = final_df.shape[0]
gold_percentage = round((gold_orders_count / total_orders) * 100)
print("\nPercentage of orders by Gold members:", gold_percentage,"%")

# Revenue by city, cuisine, membership
city_cuisine_membership = final_df.groupby(['city','cuisine','membership'])['total_amount'].sum().sort_values(ascending=False)
print("\nRevenue by city, cuisine, membership combination:")
print(city_cuisine_membership.head(10))

# Revenue by quarter
final_df['quarter'] = final_df['order_date'].dt.quarter
quarter_revenue = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
print("\nRevenue by quarter:")
print(quarter_revenue)

# %%
# 5️⃣ Save cleaned final dataset (optional)
final_df.to_csv("final_food_delivery_dataset_cleaned.csv", index=False)

# %% [markdown]
# **Notebook Complete.**
# All questions are answered and final dataset saved.  
# Ready for GitHub upload.


Total orders by Gold members: 4987
Total revenue in Hyderabad: 1889367
Distinct users with orders: 2883
Average order value for Gold members: 797.15
Orders for restaurants with rating >= 4.5: 3374
Top revenue city among Gold members: Chennai
Orders in top revenue city (Gold members): 1337

Revenue by membership + cuisine combination:
membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

Cuisine stats (fewest restaurants but high revenue):
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Italian                   126     2024203.80
Indian                    126     1971412.58
Mexican                   128     2085503.09

Restaurant w